<a href="https://colab.research.google.com/github/justxoai/DeepLearning2024/blob/main/YOLOv8_VS_YOLOv11_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Essential packages and libraries

Explanation
1. `roboflow` is where we download the dataset.
2. `ultralytics` provides YOLO models. In this project, we will be using a YOLOv8 model and a YOLOv11 model.

In [ ]:
!pip install roboflow
from roboflow import Roboflow

!pip install -q ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.3/881.3 kB 23.4 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Tools for file and system operations

In [ ]:
import os
import shutil

(Not training related) Delete auto-generated sample data by google colab

In [ ]:
shutil.rmtree('sample_data', ignore_errors=True)

# Download dataset
Our dataset with a total of 343 images has already been splitted into 3 subsets, namely `train`, `valid` and `test`.
- **train set** and **valid set** (70% and 20% respectively) will be used to train the model
- **test set** (10%) will be used after the training process to compare actual results with the predicted results given by the model

Download and refactor folder names of dataset

In [ ]:
rf = Roboflow(api_key="???") # Omitted for security reasons
project = rf.workspace("kfupm-7hvnm").project("lung-tumor-2xcgn")
version = project.version(3)

current_folder_name = 'Lung-Tumor-3'
dataset_name = 'dataset'

v8seg_dataset_obj = version.download("yolov8")
os.rename(current_folder_name, dataset_name)
print(f"\nFolder renamed from {current_folder_name} to {dataset_name}")

os.rename('/content/' + dataset_name + '/data.yaml', '/content/' + dataset_name + '/v8seg_data.yaml')

v11seg_dataset_obj = version.download("yolov11")
shutil.move('/content/' + current_folder_name + '/data.yaml', '/content/' + dataset_name + '/v11seg_data.yaml')
shutil.rmtree(current_folder_name)

v8seg_dataset_path = os.path.join('/content/' + dataset_name, 'v8seg_data.yaml')
v11seg_dataset_path = os.path.join('/content/' + dataset_name, 'v11seg_data.yaml')

def replace_path_in_file(path: str):
    with open(path, 'r') as file:
        content = file.read()
    content = content.replace(current_folder_name, '..')
    with open(path, 'w') as file:
        file.write(content)

replace_path_in_file(v8seg_dataset_path)


loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.3.1, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Lung-Tumor-3 in yolov8:: 100%|██████████| 698/698 [00:00<00:00, 1536.38it/s]



Folder renamed from Lung-Tumor-3 to dataset



Extracting Dataset Version Zip to Lung-Tumor-3 in yolov11:: 100%|██████████| 698/698 [00:00<00:00, 1456.72it/s]


Delete dataset folder if needed to re-download

In [ ]:
shutil.rmtree('dataset')

# Training

In [ ]:
def start_train(model, dataset_path, output_folder_name):
    return model.train(
        data = dataset_path,
        project = output_folder_name,
        name = '100_epochs',
        epochs = 100,
        patience = 10,
        batch = 4,
        imgsz = 640,
        workers = 8
    )

## Training Model Parameters
In this topic, we are able to pass a number of arguments:

- **`data:`** Data configuration

    -> This parameter points to where the training data is stored, including both images and their corresponding labels.

- **`project:`** Output directory

    -> This parameter sets the project folder where training results (such as model weights and evaluation metrics) will be saved.

- **`name:`** Subfolder

    -> A subfolder within the project folder will be created with this name to store the results of this specific training run.

- **`epochs:`** Define the number of training epochs

    -> An epoch refers to one iteration over the entire training dataset. The more epochs, the more opportunities the model has to learn from the data. However, too many epochs can lead to overfitting.

- **`patience:`**

    -> The patience parameter controls how many epochs the training will continue after a metric (like validation loss) stops improving. It helps prevent overtraining and overfitting.

- **`batch:`** Determine batch size  

    -> Batch size defines how many samples will be processed before the model's weights are updated. Larger batch sizes may result in more stable training but require more memory (especially on GPUs). Smaller batch sizes can reduce memory usage but might introduce noise in the weight updates, which could affect convergence.

- **`imgsz:`** Define input image size  

    -> This specifies the size (in pixels) of the images that will be fed into the model. In this case, images will be resized to 640x640 before training. Larger images provide more detail but require more memory and computation time. Smaller images may speed up training but at the potential cost of losing important information.

- **`workers:`**

    -> Refers to the number of workers (parallel processes) used to load the data. More workers can speed up data loading during training.

### YOLOv8

- The line `v8seg_model = YOLO("yolov8x-seg.pt")` is creating an instance of the YOLO model using a specific pre-trained model file, in this case, `yolov8x-seg.pt`.
  - `yolov8x`: YOLO version 8 model in its "extra large" (`x`) configuration
    - **65,193,619 parameters**
    - **313 layers**
  - `seg`: The model is specifically trained for segmentation tasks
- `v8seg_dataset_path`: The path to its dataset configuration will be `/content/dataset/v8seg_data.yaml`
- `v8seg_out`: The name of project folder

In [ ]:
v8seg_model = YOLO("yolov8x-seg.pt")
v8seg_out = 'v8seg_out'
v8seg_results = start_train(v8seg_model, v8seg_dataset_path, v8seg_out)

100%|██████████| 137M/137M [00:01<00:00, 86.3MB/s]


Ultralytics 8.3.1 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8x-seg.pt, data=/content/dataset/v8seg_data.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=v8seg_out, name=100_epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning /content/dataset/train/labels.cache... 240 images, 0 backgrounds, 0 corrupt: 100%|██████████| 240/240 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/dataset/valid/labels.cache... 69 images, 0 backgrounds, 0 corrupt: 100%|██████████| 69/69 [00:00<?, ?it/s]


...

     31/100      10.9G       1.75      1.512     0.8847      1.235          3        640: 100%|██████████| 60/60 [01:03<00:00,  1.07s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:06<00:00,  1.31it/s]

                   all         69         70       0.93      0.957      0.952      0.424      0.875        0.9      0.919      0.357
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 21, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



31 epochs completed in 0.802 hours.
Optimizer stripped from v8seg_out/100_epochs/weights/last.pt, 130.9MB
Optimizer stripped from v8seg_out/100_epochs/weights/best.pt, 130.9MB

Validating v8seg_out/100_epochs/weights/best.pt...
Ultralytics 8.3.1 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x-seg summary (fused): 313 layers, 65,193,619 parameters, 0 gradients, 313.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.28it/s]


                   all         69         70      0.903      0.928      0.925      0.415       0.94      0.894      0.908      0.385
Speed: 0.7ms preprocess, 85.5ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to v8seg_out/100_epochs


Delete project folder if needed to do another train

In [ ]:
shutil.rmtree('v8seg_out')

### YOLO11

- The line `v11seg_model = YOLO("yolo11x-seg.pt")` is creating an instance of the YOLO model using a specific pre-trained model file, in this case, `yolo11x-seg.pt`.
  - `yolo11x`: YOLO version 11 model in its "extra large" (`x`) configuration
    - **62,003,283 parameters**
    - **491 layers**
  - `seg`: The model is specifically trained for segmentation tasks
- `v11seg_dataset_path`: The path to its dataset configuration will be `/content/dataset/v11seg_data.yaml`
- `v11seg_out`: The name of project folder

In [ ]:
v11seg_model = YOLO("yolo11x-seg.pt")
v11seg_out = 'v11seg_out'
v11seg_results = start_train(v11seg_model, v11seg_dataset_path, v11seg_out)

100%|██████████| 119M/119M [00:03<00:00, 38.3MB/s]


Ultralytics 8.3.1 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolo11x-seg.pt, data=/content/dataset/v11seg_data.yaml, epochs=100, time=None, patience=10, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=v11seg_out, name=100_epochs, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

100%|██████████| 755k/755k [00:00<00:00, 25.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2784  ultralytics.nn.modules.conv.Conv             [3, 96, 3, 2]                 
  1                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  2                  -1  2    389760  ultralytics.nn.modules.block.C3k2            [192, 384, 2, True, 0.25]     
  3                  -1  1   1327872  ultralytics.nn.modules.conv.Conv             [384, 384, 3, 2]              
  4                  -1  2   1553664  ultralytics.nn.modules.block.C3k2            [384, 768, 2, True, 0.25]     
  5                  -1  1   5309952  ultralytics.nn.modules.conv.Conv             [768, 768, 3, 2]              
  6                  -1  2   5022720  ultralytics.nn.modules.block.C3k2            [768, 768, 2, True]           
  7                  -1  1   5309952  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 111MB/s]

...

     28/100      10.4G      1.856      1.534     0.9229      1.276          6        640: 100%|██████████| 60/60 [01:10<00:00,  1.17s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.27it/s]

                   all         69         70      0.944      0.871      0.891      0.407      0.867        0.8      0.783      0.264
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 18, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



28 epochs completed in 0.719 hours.
Optimizer stripped from v11seg_out/100_epochs/weights/last.pt, 124.8MB
Optimizer stripped from v11seg_out/100_epochs/weights/best.pt, 124.8MB

Validating v11seg_out/100_epochs/weights/best.pt...
Ultralytics 8.3.1 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11x-seg summary (fused): 491 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.19it/s]


                   all         69         70      0.949      0.814      0.913      0.413      0.927      0.786      0.906      0.353
Speed: 0.3ms preprocess, 90.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to v11seg_out/100_epochs


Delete project folder if needed to do another train

In [ ]:
shutil.rmtree('v11seg_out')

# Post Training Activities

## Draw Actual Images from **Test set**
Images and labels are provided by the dataset.

In [ ]:
import cv2
import numpy as np
import os

def draw_actual(images_folder, labels_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all image files in the images folder
    for image_file in os.listdir(images_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
            # Load the image
            image_path = os.path.join(images_folder, image_file)
            image = cv2.imread(image_path)

            # Load the segmentation label
            label_path = os.path.join(labels_folder, os.path.splitext(image_file)[0] + '.txt')
            height, width, _ = image.shape

            # Check if label file exists
            if not os.path.exists(label_path):
                print(f"Label file {label_path} does not exist. Skipping...")
                continue

            with open(label_path, 'r') as file:
                labels = file.readlines()

            # Iterate through each object in the label
            for label in labels:
                label = label.strip().split()
                class_id = int(label[0])
                polygon_coords = list(map(float, label[1:]))

                # Reshape polygon coordinates
                polygon = np.array(polygon_coords).reshape(-1, 2)
                polygon[:, 0] = polygon[:, 0] * width  # Convert normalized x to pixel x
                polygon[:, 1] = polygon[:, 1] * height # Convert normalized y to pixel y
                polygon = polygon.astype(np.int32)

                # Draw the polygon mask on the image (for visualization)
                cv2.polylines(image, [polygon], isClosed=True, color=(0, 0, 255), thickness=2)
                cv2.fillPoly(image, [polygon], color=(255, 0, 255))  # Draw the filled mask

            # Save the modified image
            output_image_path = os.path.join(output_folder, image_file)
            save_success = cv2.imwrite(output_image_path, image)

            if save_success:
                print(f"Image saved successfully at {output_image_path}")
            else:
                print(f"Failed to save the image at {output_image_path}")

In [ ]:
images = "/content/dataset/test/images"
labels = "/content/dataset/test/labels"
output = "/content/dataset/test/actual"

draw_actual(images, labels, output)

Image saved successfully at /content/dataset/test/actual/003380_03_01_140_png_jpg.rf.f79c35e472e03b850cd573ff0dd8b253.jpg
Image saved successfully at /content/dataset/test/actual/000103_13_01_019_png_jpg.rf.9ab571abaf8f3b70b259dec2ba63f4f5.jpg
Image saved successfully at /content/dataset/test/actual/000306_06_02_126_png_jpg.rf.38ae65457b77dc15f7ed15d72984b396.jpg
Image saved successfully at /content/dataset/test/actual/000103_08_01_016_png_jpg.rf.6fff7bdb9e44d43c7c96680912780e93.jpg
Image saved successfully at /content/dataset/test/actual/000874_03_02_219_png_jpg.rf.1447e1d2cca9a1bd99d429f6a06f4648.jpg
Image saved successfully at /content/dataset/test/actual/001404_01_01_139_png_jpg.rf.286b22d8ea74a86732ab9397d3553be4.jpg
Image saved successfully at /content/dataset/test/actual/000103_02_01_060_png_jpg.rf.e45c32f9edd4abfe3b94915ccbb256fd.jpg
Image saved successfully at /content/dataset/test/actual/002265_02_01_031_png_jpg.rf.cac17609de187598bd537354b0ce3e75.jpg
Image saved successfully

## Draw Predicted Images from **Test set**
Images are provided by the dataset, but the labels will be predicted by our previously trained model

In [ ]:
import cv2
import os

def draw_predicted(images_folder, trained_model, output_folder):

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Iterate through all image files in the images folder
    for image_file in os.listdir(images_folder):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):  # Check for image file extensions
            # Load the image
            image_path = os.path.join(images_folder, image_file)
            image = cv2.imread(image_path)

            # Predict with the model
            results = trained_model(image_path)  # Predict on the image file

            # Iterate through the results
            for result in results:
                masks = result.masks  # Get masks from results
                names = trained_model.names  # Get class names from the model

                # Draw the masks on the image
                if masks is not None and masks.data is not None:
                    for i in range(masks.data.shape[0]):  # Iterate over each mask
                        mask_array = masks.data[i].cpu().numpy()  # Get mask data as NumPy array
                        mask_array = (mask_array > 0).astype(np.uint8)  # Binarize the mask

                        # Create a red color version of the mask
                        colored_mask = np.zeros_like(image)  # Create an empty image with the same shape
                        colored_mask[mask_array == 1] = [0, 255, 255]  # Set the mask region to red

                        # Blend the mask with the original image
                        alpha = 0.5  # Adjust transparency
                        blended = cv2.addWeighted(image, 1 - alpha, colored_mask, alpha, 0)

                        # Update the original image with the blended mask
                        image = blended

                        # Access confidence and class using boxes
                        if result.boxes is not None:
                            conf = result.boxes[i].conf.item()  # Confidence score
                            cls = result.boxes[i].cls.item()  # Class index
                            class_name = names[int(cls)]  # Get class name from index

                            # Calculate the position to draw the text (using mask's centroid)
                            y, x = np.where(mask_array == 1)  # Get coordinates of the mask
                            if len(x) > 0 and len(y) > 0:  # Check if there are any coordinates
                                text_x, text_y = int(x.mean()), int(y.mean())  # Get the centroid for text position
                                # Draw the class name and confidence next to the mask
                                cv2.putText(image, f'{class_name}: {conf:.2f}',
                                            (text_x, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                                            1, (255, 255, 0), 2)

            # Save the modified image
            output_image_path = os.path.join(output_folder, image_file)
            save_success = cv2.imwrite(output_image_path, image)

            if save_success:
                print(f"Image saved successfully at {output_image_path}")
            else:
                print(f"Failed to save the image at {output_image_path}")

In [ ]:
images = "/content/dataset/test/images"

# YOLOv8
model = YOLO("/content/v8seg_out/100_epochs/weights/best.pt")
output = "/content/dataset/test/v8predict"
draw_predicted(images, model, output)

# YOLO11
model = YOLO("/content/v11seg_out/100_epochs/weights/best.pt")
output = "/content/dataset/test/v11predict"
draw_predicted(images, model, output)


image 1/1 /content/dataset/test/images/003380_03_01_140_png_jpg.rf.f79c35e472e03b850cd573ff0dd8b253.jpg: 640x640 1 Tumor, 110.5ms
Speed: 2.0ms preprocess, 110.5ms inference, 10.0ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/dataset/test/v8predict/003380_03_01_140_png_jpg.rf.f79c35e472e03b850cd573ff0dd8b253.jpg

image 1/1 /content/dataset/test/images/000103_13_01_019_png_jpg.rf.9ab571abaf8f3b70b259dec2ba63f4f5.jpg: 640x640 1 Tumor, 67.0ms
Speed: 1.9ms preprocess, 67.0ms inference, 3.9ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/dataset/test/v8predict/000103_13_01_019_png_jpg.rf.9ab571abaf8f3b70b259dec2ba63f4f5.jpg

image 1/1 /content/dataset/test/images/000306_06_02_126_png_jpg.rf.38ae65457b77dc15f7ed15d72984b396.jpg: 640x640 2 Tumors, 69.0ms
Speed: 1.7ms preprocess, 69.0ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)
Image saved successfully at /content/dataset/test/v8predict/00030

# File Management
NOT related to training

## Duplicate project folder with no weights included

In [ ]:
def copy_folder_contents(src_folder, dest_folder):
    # Check if the source folder exists
    if not os.path.exists(src_folder):
        print(f"Source folder '{src_folder}' does not exist.")
        return

    # Create destination folder if it doesn't exist
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    # Iterate through all the files and directories in the source folder
    for item in os.listdir(src_folder):
        src_path = os.path.join(src_folder, item)
        dest_path = os.path.join(dest_folder, item)

        # Copy files and directories
        if os.path.isdir(src_path):
            shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
        else:
            shutil.copy2(src_path, dest_path)

    print(f"All contents copied from '{src_folder}' to '{dest_folder}'.")


In [ ]:
v8seg_noweights_path = '/content/v8seg_noweights/'
v11seg_noweights_path = '/content/v11seg_noweights/'
os.makedirs(v8seg_noweights_path)
os.makedirs(v11seg_noweights_path)

src_folder = '/content/v8seg_out/'
copy_folder_contents(src_folder, v8seg_noweights_path)

src_folder = '/content/v11seg_out/'
copy_folder_contents(src_folder, v11seg_noweights_path)

shutil.rmtree(v8seg_noweights_path + '100_epochs/weights')
shutil.rmtree(v11seg_noweights_path + '100_epochs/weights')

All contents copied from '/content/v8seg_out/' to '/content/v8seg_noweights/'.
All contents copied from '/content/v11seg_out/' to '/content/v11seg_noweights/'.


## Save progress to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_dir = '/content/dataset'
v8seg_out_path = '/content/v8seg_out'
v11seg_out_path = '/content/v11seg_out'
v8seg_noweights_path = '/content/v8seg_noweights'
v11seg_noweights_path = '/content/v11seg_noweights'

drive_dir = "/content/drive/MyDrive/DL"

shutil.make_archive(dataset_dir, 'zip', dataset_dir)
shutil.make_archive(v8seg_out_path, 'zip', v8seg_out_path)
shutil.make_archive(v11seg_out_path, 'zip', v11seg_out_path)
shutil.make_archive(v8seg_noweights_path, 'zip', v8seg_noweights_path)
shutil.make_archive(v11seg_noweights_path, 'zip', v11seg_noweights_path)

shutil.move("/content/dataset.zip", os.path.join(drive_dir, "dataset.zip"))
shutil.move("/content/v8seg_out.zip", os.path.join(drive_dir, "v8seg_out.zip"))
shutil.move("/content/v11seg_out.zip", os.path.join(drive_dir, "v11seg_out.zip"))
shutil.move("/content/v8seg_noweights.zip", os.path.join(drive_dir, "v8seg_noweights.zip"))
shutil.move("/content/v11seg_noweights.zip", os.path.join(drive_dir, "v11seg_noweights.zip"))

print("Files moved successfully to Google Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Files moved successfully to Google Drive.


## Restore progress from Drive

In [ ]:
shutil.copy('/content/drive/MyDrive/DL/dataset.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/v8seg_out.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/v11seg_out.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/v8seg_noweights.zip', '/content/')
shutil.copy('/content/drive/MyDrive/DL/v11seg_noweights.zip', '/content/')

shutil.unpack_archive('dataset.zip', '/content/dataset/')
shutil.unpack_archive('v8seg_out.zip', '/content/v8seg_out/')
shutil.unpack_archive('v11seg_out.zip', '/content/v11seg_out/')
shutil.unpack_archive('v8seg_noweights.zip', '/content/v8seg_noweights/')
shutil.unpack_archive('v11seg_noweights.zip', '/content/v11seg_noweights/')

'/content/obb_project.zip'